# Web Scraping: Extrayendo datos de Internet

## Introducción a HTML

El lenguaje principal de la internet es HTML, cuando nosotros vemos algo así

![](multimedia/hello-world.jpeg)

Eso se genera a partir de una código que luce así

```
<html>
<header><title>Web Scraping - Instituto Humai</title></header>
<body>
<h1>¡Hola!</h1>
<p>Esto es un sitio web</p>
</body>
</html>
```

**_Nota_**: Para saber más sobre HTML podés consultar [acá](https://www.w3schools.com/TAGS/default.ASP) la lista de etiquetas de este lenguaje.

## ¿Cómo consigo el código HTML?

Ahora que sabemos cuál es el componente principal de los sitios webs podemos intentar programar a nuestra computadora para leer HTML y extraer información útil.

Para conseguir el código de un sitio web podemos presionar ctrl+u en el navegador.

Para hacer lo mismo desde python se hace lo siguiente

In [1]:
#Importamos la libreria necesaria
import requests

un_sitio_web = "https://es.wikipedia.org/wiki/HTML"

# esto descarga la información del sitio web
# similarmente a lo que hace un navegador web antes de mostrarnos el contenido de forma mas amigable para un humano
resultado = requests.get(un_sitio_web)

# accedemos al código yendo al atributo "text" del resultado
codigo_html = resultado.text
codigo_html

'\n<!DOCTYPE html>\n<html class="client-nojs" lang="es" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>HTML - Wikipedia, la enciclopedia libre</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\\t.","\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"],"wgRequestId":"54724279-5e95-4537-a46a-9bb77517c0b3","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"HTML","wgTitle":"HTML","wgCurRevisionId":127943637,"wgRevisionId":127943637,"wgArticleId":1366,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:Artículos con datos por trasladar a Wikidata","Wikipedia:Artículos buenos en la Wikipedia en ucraniano","Wikipedia:Artículos con identificador

### ¿Qué acabo de hacer?

Veamos algunos detalles más sobre cómo descargar el contenido de un sitio web (O cómo se le suele decir en la jerga de la programación _realizar un request_). Como dijimos, en python se puede utilizar la función get de la libreria requests para hacer esto, veamos con mayor profundidad cómo se utiliza.

In [2]:
url = 'http://www.laprensa.com.ar/'

headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
resp = requests.get(url, headers = headers)

A parte de la _url_, muchas veces se especifican los _headers_, estos son objetos que proveen datos sobre nuestro _request_, por ejemplo en el campo user-agent brindamos detalles sobre quienes somos (Nuestro sistema operativo, navegador web y demás). En este caso, como no estamos usando un navegador sino que hacemos el _request_ desde Python normalmente se omite este campo, o en caso de ser obligatorio se puede inventar, ya que algunos sitios nos van a ignorar a menos que especifiquemos este campo.

- Consultas
    - ¿Por qué los sitios te podrían bloquear/ignorar?
    - ¿De donde saco un user-agent?

Como vimos antes la función get retorna un objeto, el cual llamamos _resp_, este es un elemento de la clase _Response_ y tiene distintos atributos a los que podemos acceder.

In [3]:
#Vemos el código de estado
# 200 es que esta todo bien, 5xx o 4xx es que esta todo mal (Por ejemplo el clasico 404)
resp.status_code

200

In [4]:
#Vemos los headers que enviamos
resp.request.headers

{'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

El atributo que nos interesa particularmente es resp.text, que guardan el contenido de la página.

Como vamos a descargar el codigo de un sitio frecuentemente armamos una funcion para no reescribir lo mismo muchas veces

In [5]:
def codigo_html(url):
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
    resp = requests.get(url, headers = headers)
    return resp.text

### Documentación
La función get y la clase Response fueron desarrolladas por lxs programadores que crearon la librería requests. Si quieren saber mas sobre algún detalle siempre es recomendable buscar en la documentación oficial de la librería.

## ¿Cómo extraigo datos útiles del código HTML?

- Veamos un ejemplo inspeccionando con chrome un sitio web


### Método 1: Expresiones regulares

RegEx para los amigos. Son un mini lenguaje de programación diseñado para realizar búsquedas en strings.

Las funciones principales de la librería re son:
- re.findall(pattern, string) para encontrar todos los resultados de una búsqueda
- re.search(pattern, string) para encontrar el primer resultado que coincida
- re.sub(pattern, replace, string) para substituir un texto por otro

#### Recursos útiles

- [Testeo de regex online](https://regex101.com/)
- [CheatSheet](https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf)


## Sintaxis para construir regex

Grupos de captura y tipos de datos

|     |                      |          |                         |
|----|-----------------------|----------|-------------------------|
| () | grupo de captura      | (?:)     | grupo de no captura     |
| \w | caracter alfanumérico | .        | cualquier cosa menos \n |
| \d | dígito                | \|       | operador "or"           |
| \s | espacio en blanco     | [m-z3-9] | rangos                  |
| [] | conjunto              |          |                         |


Cuantificadores


|      |                                              |
|------|----------------------------------------------|
| +    | Uno o más del elemento anterior              |
| *    | Cero o más del elemento anterior             |
| {4,} | Cuatro o más del elemento anterior           |
| ?    | Cambia el operador anterior de lazy a greedy |

In [6]:
# python utiliza la libreria llamada re para todo lo relacionado a regular expressions
import re

comentario_de_mercadolibre = 'hola soy @mariadominguez, me interesa el producto, te dejo mi celu 1565525233, saludos'

def encontrar_telefonos(texto):
    regla_de_busqueda = r'(15[0-9]{8})'
    return re.findall(regla_de_busqueda, texto)

def encontrar_usuarios(texto):
    regla_de_busqueda = r'@([a-zA-Z]+)'
    return re.findall(regla_de_busqueda, texto)

print(encontrar_telefonos(comentario_de_mercadolibre))
print(encontrar_usuarios(comentario_de_mercadolibre))

['1565525233']
['mariadominguez']


#### Ejercicio

Usa regex para hacer una función que busque todos los emails en un texto

In [7]:
def encontrar_emails(texto):
    # COMPLETAR
    regla_de_busqueda = r"([a-z0-9_]+@[a-z]+\.[a-z]+)"
    # SOLUCION: regla_de_busqueda = r"([a-z0-9_]+@[a-z]+\.[a-z]+)"
    return re.findall(regla_de_busqueda,texto)

texto = "Hola te paso mi mail python@hotmail.com, saludos. Si no te funciona mandame a este otro, pedro_2010@yahoo.com"
encontrar_emails(texto)

['python@hotmail.com', 'pedro_2010@yahoo.com']

#### Aplicandolo a la web
##### Ejemplo 1: Usamos regex para extraer los títulos del diario La Prensa.


```html
<h2 class="entry__title"><a href="http://www.laprensa.com.ar/491843-Dilemas-de-la-batalla-cultural-I.note.aspx" target="_self" onclick="javascript:if(typeof(_gaq)!='undefined'){_gaq.push(['_trackEvent', 'Notas', 'Cultura', 'Dilemas de la batalla cultural (I)'])};">Dilemas de la batalla cultural (I)</a></h2>
```


In [8]:
#Usamos el navegador para identificar la estructura de los datos que queremos extraer y creamos el patrón de búsqueda
regla_de_busqueda = r';">(.+)</a></h2>'

In [9]:
#Usamos findall para encontrar todas las coincidencias
import re
titles = [m for m in re.findall(regla_de_busqueda, codigo_html("http://www.laprensa.com.ar/"))]

In [10]:
titles

['Deuda: un respiro en medio de un deterioro sin precedentes',
 'Dilemas de la batalla cultural (II)',
 'Cupos, cupos y más cupos, el sexo del absurdo',
 'El campo digital se expone en San Nicolás',
 'Gracias al Bocha, el campeonato se tiñe de rojo']

#### Ejercicio

#### Modifiquen la regla de búsqueda para que descargue los links a las notas en vez del título

### Método 2: BeautifulSoup
Esta librería provee un parser de html, o sea un programa que entiende el código, permitiendonos hacer consultas mas sofisticadas de forma simple, por ejemplo "buscame todos los titulos h2 del sitio".

In [11]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(codigo_html("http://www.laprensa.com.ar/"), 'html.parser')
for title in soup.find_all("h2",class_="entry__title"):
    print(title.find("a").text)

Deuda: un respiro en medio de un deterioro sin precedentes
Dilemas de la batalla cultural (II)
Cupos, cupos y más cupos, el sexo del absurdo
Pandemia: “Un 98% de los positivos apenas cursa un resfrío común”

El campo digital se expone en San Nicolás
Gracias al Bocha, el campeonato se tiñe de rojo
Un fiel lector de La Prensa: Juan Carlos Arias Divito


#### Ejemplo 2: Cortazar

In [12]:
url = 'http://ciudadseva.com/autor/julio-cortazar/cuentos/'
import re
response = requests.get(url)
codigo_html_crudo = response.text

In [13]:
for s_url in re.findall(r'(https://ciudadseva.com/texto/.+/)', codigo_html_crudo):
    cuento = requests.get(s_url)
    codigo_html_interpretado = BeautifulSoup(cuento.content, 'html.parser')
    for elem in codigo_html_interpretado.findAll("div", { "class" : "text-justify" })[:2]:
        cuento = elem.text
    
    # Asi podemos guardar los resultados
    nombre_del_archivo = s_url.split('/')[-2]
    with open (f"multimedia/cortazar/{nombre_del_archivo}.txt", 'w') as out:
        out.write(cuento)

## Práctica: Mercadolibre

Descargá y calculá el promedio de los precios que aparecen en la primer página de mercado libre al buscar gibson

In [28]:
import requests
import re

def precios_gibson():
    url = "https://listado.mercadolibre.com.ar/gibson"
    soup = BeautifulSoup(codigo_html(url), 'html.parser')
    prices = []
    # COMPLETAR
    # SOLUCION:
    '''
    for price in soup.find_all("span",class_="price__fraction"):
        price = price.text
        # Los precios originalmente son strings: "101.324"
        # Los convierto a float (numero) de esta manera
        price = float(price.replace(".",""))
        prices.append(price)
    '''

    return prices

precios = precios_gibson()
import numpy as np
print(f"El precio promedio es {int(np.nanmean(precios))}")

## robots.txt + sitemap (legalidades guiño guiño)